# Scikit-learn VS. Statsmodels 

- **scikit-learn library**: 
    - It is primarily used for machine learning and predictive modeling. 
    - It provides a simple and efficient tool for data mining and data analysis.

- Input: For example, the model is initialized with LinearRegression(), and the data is fitted with lm.fit().
- Output: After fitting, the main outputs are the coefficients (lm.coef_) and intercept (lm.intercept_). 

- It provides tools for making predictions (lm.predict()) but less detailed statistical summaries.


- **statsmodels**: 
    - It focused on statistical modeling. 
    - It provides more detailed output related to the statistical properties of the model, making it useful for inferential statistics.
- Input: For example, the model is initialized with sm.OLS(y, X), and the data is fitted with model.fit().
- Output: The results object provides a comprehensive summary (results.summary()) including coefficients, standard errors, t-values, p-values, R^2, adjusted R^2, F-statistic, and more. 

- It’s designed to give a full statistical report.

We can integrate both into your workflow: using scikit-learn for prediction and statsmodels for detailed statistical analysis. 


## **Classification Models VS. Regression Models**

`Classification Models`

**Nature of Output:**

- Discrete Labels: Classification models predict categorical outcomes. For example, determining whether an email is spam or not spam (binary classification) or classifying an image into one of several categories (multiclass classification).

**Goal:**

- Class Membership: The primary goal is to assign input data to one of several predefined categories or classes.

**Common Algorithms:**

- Logistic Regression: Used for binary classification problems.
- Decision Trees and Random Forests: Can be used for both binary and multiclass classification.
- Support Vector Machines (SVM): Effective for high-dimensional spaces.
- K-Nearest Neighbors (KNN): A simple, instance-based learning algorithm.
- Gradient Boosting Machines (GBM): An ensemble technique that builds models sequentially.

**Evaluation Metrics:**

- Accuracy: The proportion of correctly predicted instances.
- Precision, Recall, F1-Score: Used especially when dealing with imbalanced datasets.
- ROC-AUC: Measures the ability of the model to distinguish between classes.

`Regression Models`

**Nature of Output:**

- Continuous Values: Regression models predict continuous numeric outcomes. For example, predicting house prices or the temperature for a given day.

**Goal:**

- Value Estimation: The primary goal is to estimate the value of the output variable based on the input features.

**Common Algorithms:**

- Linear Regression: A fundamental and widely used method for predicting continuous values.
- Ridge and Lasso Regression: Extensions of linear regression that include regularization.
- Decision Trees and Random Forests: Can also be used for regression tasks.
- Support Vector Regression (SVR): An extension of SVM for regression problems.
- Gradient Boosting Regressors: Similar to classification, but tailored for continuous outcomes.

**Evaluation Metrics:**

- Mean Squared Error (MSE): The average of the squared differences between the predicted and actual values.
- Root Mean Squared Error (RMSE): The square root of the MSE, providing error in the same units as the output variable.
- Mean Absolute Error (MAE): The average of the absolute differences between the predicted and actual values.
- R-squared: Indicates the proportion of the variance in the dependent variable that is predictable from the independent variables.


In [4]:
###### scikit-learn/regression ######
# 0. Import Required Libraries
# 1. Visualize dataset according to data characteristics
# 2. Load the Dataset (Boston dataset in this case)
# 3. Split the Data
# 4. Define Regression Pipelines with hyperparams
# 5. Train Pipelines with grid search
# 6. Return the Best Model and Visualize Performance Metrics
# 7. Predicting with the best model

# Please note: the above steps are not strictly in such an order, 
# steps may overlap each other.

# The code already includes as many models as possible, 
# but specific adjustments will need to be made based on the project.
###### scikit-learn/regression ######

In [1]:
# 0. Import Required Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.model_selection import learning_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import validation_curve
from ISLP import load_data
import numpy as np
import pandas as pd
from pprint import pprint
import warnings # for muting warning messages
# mute warning messages
warnings.filterwarnings('ignore')
from sklearn.pipeline import make_pipeline
from joblib import dump
from joblib import load
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV


In [ ]:
# 1. Visualize dataset according to data characteristics
# 1.1 Subfunctions to plot for dataset
def plot_all_data(df,df_without_target, target_variable,model, feature_names):

    # visualize_distribution(df_without_target)
    visualize_correlation_heatmap(df)
    # visualize_scatter_matrix(df, target_variable)
    # Identify categorical features
    categorical_features = df.select_dtypes(include=['object', 'category']).columns
    print("Categorical features:", categorical_features)
    
    if len(categorical_features) > 0:
        for categorical_feature in categorical_features:
            visualize_boxplots(df, target_variable, categorical_feature)
    else:
        print("No categorical features found for boxplots.")

    visualize_pairwise_scatter_plots(df)
    visualize_feature_importance(model, feature_names)
    
# Histograms and Density Plots
# Each histogram represents the distribution of values within a specific feature.
def visualize_distribution(df):
    df.hist(bins=20, figsize=(15, 10))
    plt.suptitle("Histograms of Features", fontsize=16)
    plt.tight_layout()
    plt.show()

# Correlation Heatmap
# It shows one feature increases, the other features increases or decreases
def visualize_correlation_heatmap(df):
    corr = df.corr()
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5)
    plt.title('Correlation Heatmap')
    plt.show()

# Scatter Plot Matrix
# The pair plot shows scatter plots for each pair of features in the DataFrame. 
# Each subplot represents the relationship between two features.
def visualize_scatter_matrix(df, target_variable):
    sns.pairplot(df.sample(frac=0.1, random_state=42), hue=target_variable, palette='bwr')
    plt.title("Pairplot of Dataset")
    plt.show()

# Boxplots
# Shows the distribution of the target variable within each category of the categorical feature.
def visualize_boxplots(df, target_variable, categorical_feature):
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=categorical_feature, y=target_variable, data=df)
    plt.title("Boxplot of Target Variable Across Categories")
    plt.show()

# Pairwise Feature Scatter Plots
def visualize_pairwise_scatter_plots(df):
    sns.pairplot(df.sample(frac=0.1, random_state=42))
    plt.title("Pairwise Scatter Plots of Features")
    plt.show()

# Feature Importance Plot
def visualize_feature_importance(model, feature_names):
    importances = None
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    elif hasattr(model, 'coef_'):
        importances = np.abs(model.coef_)
    
    if importances is not None and len(importances) > 0:
        # Create a DataFrame to sort and display feature importances
        feature_importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importances
        })

        # Sort the features by importance
        feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

        # Plot the feature importances
        plt.figure(figsize=(10, 6))
        sns.barplot(x='importance', y='feature', data=feature_importance_df)
        plt.title("Feature Importance Plot")
        plt.xlabel("Importance")
        plt.ylabel("Feature")
        plt.show()


# Geospatial Visualization (if applicable)
# Time Series Plots (if applicable)
# Interactive Plots (if applicable)

# 1.2 Use the above subfunctions to plot for dataset
# Load dataset
try:
    dataset_name = 'boston'
    data = load_dataset(dataset_name)
except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")
    exit()

# Select numeric columns only
df = data.select_dtypes(include=['float64', 'int64'])
target_variable='mpg'
df_without_target=df.drop(columns=target_variable)

plot_all_data(df,df_without_target,target_variable,model, feature_names)



In [16]:
# Model fitting
def fit_regression_models(X,y,testsize,degree_list=None, num_bootstrap_samples=50, confidence = 0.95, random_seed=42,top_importances=10, model_step_name="regression_model"):
    # 3. Split the Data
    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testsize, random_state=random_seed)
    except Exception as e:
        print(f"An error occurred while splitting the data: {e}")
        exit()
    column_names=X_train.columns.tolist()
    target_variable=y_train.name
    # 4. Define Regression Models
    regression_models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(),
        'Lasso Regression': Lasso(),
        'ElasticNet Regression': ElasticNet(),
        'Decision Tree Regressor': DecisionTreeRegressor(),
        'Random Forest Regressor': RandomForestRegressor(),
        'Gradient Boosting Regressor': GradientBoostingRegressor(),
        'Support Vector Regressor': SVR(),
        'K-Neighbors Regressor': KNeighborsRegressor(),
        'Neural Network Regressor': MLPRegressor(max_iter=1000)  # Increased max_iter to ensure convergence
    }
    model_params = {
        'Random Forest Regressor': {
            'n_estimators': [50, 100, 200], 
            'max_features': ['auto', 'sqrt', 'log2'], 
            'max_depth': [None, 10, 20, 30], 
            'min_samples_split': [2, 5, 10], 
            'min_samples_leaf': [1, 2, 4]
        },
        'Gradient Boosting Regressor': {
            'n_estimators': [50, 100, 200], 
            'learning_rate': [0.01, 0.1, 0.5, 1.0], 
            'max_depth': [3, 4, 5], 
            'min_samples_split': [2, 5, 10], 
            'min_samples_leaf': [1, 2, 4], 
            'subsample': [0.8, 0.9, 1.0]
        }
    }
   
    # Add polynomial regression models with each degree in the degree_list
    if degree_list is None:
        degree_list=[1] # Default to linear if no degrees are specified
    else: # add 1 into the list for later loop
        degree_list = list(degree_list)
        if 1 not in degree_list:
            degree_list.append(1)
    # After appending, the degree_list wont be empty

    # Function to create a pipeline with poly features
    # When degree>1 apply pipelines to those models which support poly
    def create_pipeline(model, degree):
        if degree > 1:
            return Pipeline([
                ("poly_features", PolynomialFeatures(degree=degree)),
                ("regression_model", model)
            ])
        else:
            return Pipeline([
                ("regression_model", model)
            ])

    regression_pipelines = {}
    for name, model in regression_models.items():    
        # Add all models into the pipelines
        for degree in degree_list:
            regression_pipelines[f'{name} (Degree {degree})'] = Pipeline([
                ("poly_features", PolynomialFeatures(degree=degree, include_bias=False)),
                (f"{model_step_name}", GridSearchCV(model, model_params[name], cv=5))
            ])
         
    # print(regression_pipelines)
    best_model_info = None
    best_model = None
    metrics_list = []

    for name, pipeline in regression_pipelines.items():
        try: 
            # 5. Train the model
            pipeline.fit(X_train, y_train)
            
            # Save the pipelines to disk
            y_pred = pipeline.predict(X_test)
            dump(pipeline, f"{name}.joblib")
            
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            residuals = y_test - y_pred

            # Calculate variance and bias using cross-validation
            variance=np.var(y_pred)
            bias=np.mean(np.abs(y_test-y_pred))

            # Estimate model complexity (number of parameters)
            if hasattr(model, 'coef_'):
                complexity = len(model.coef_)
            elif hasattr(model, 'n_estimators'):
                complexity = model.n_estimators
            elif hasattr(model, 'alpha'):
                complexity = model.alpha
            else:
                complexity = None

            # Determine the degree for regression models
            import re
            match = re.search(r'Degree (\d+)', name)
            degree = int(match.group(1))

            # Store results for comparison
            metrics_list.append({
                'model': name,
                "pipeline":pipeline,
                'degree': degree, 
                'r2': r2,
                'mse': mse,
                'rmse': rmse,
                'mae': mae,
                'residuals_mean': residuals.mean(),
                'residuals_std': residuals.std(),
                'variance': variance,
                'bias': bias,
                'complexity': complexity
            })

        except Exception as e:
            print(f"An error occurred while training and evaluating the {name} model: {e}")

    # Sort the metrics by R² in descending order and other metrics in ascending order
    metrics_list.sort(key=lambda x: (-x['r2'], x['mse'], x['rmse'], x['mae'], x['residuals_mean'],x['residuals_std'], x['variance'],x['bias'],x['complexity']))
    # df_metrics=pd.DataFrame(metrics_list)
    # print(df_metrics)
    print(metrics_list)

    # 6. Return the best model
    best_model_info=metrics_list[0]
    best_model_name =best_model_info['model']
    best_model = best_model_info['pipeline'] 
    degree = best_model_info['degree']
    print(f'Best model found: {best_model_name}')
    # print(best_model_info)

    # Load the pipe line before predict
    # best_model=load(f"{best_model_name}.joblib")

    # Delete all other joblib files
    for name, pipeline in regression_pipelines.items():
        if name != best_model_name:
            joblib_file = f'{name}.joblib'
            if os.path.exists(joblib_file):
                os.remove(joblib_file)
    
    # 6.1 (Optional) Calculate the prediction_intervals based on the best_model
    # Resample on the train data to get sample data setS using bootstrap 
    # keep fitting with the same model type and generating new model each time
    # calculate prediction intervals
    bootstrap_predictions =[]
    prediction_intervals = []

    # In scikit-learn, the fit method is used to train a model, 
    # but it does not return a result object like in statsmodels. 
    # Instead, the model instance itself is updated with the learned parameters. 
    # One way to train the same model class is to clone the model type of the best_model  
    # on multiple sample datasets
    
    # We do not need to manually fit and transform X_train if we use a pipeline. 
    # The Pipeline object in scikit-learn takes care of applying the fit and transform methods for each step in the pipeline. 
    # When we call fit on the pipeline, it will sequentially call fit and transform for each step, passing the transformed data to the next step. Similarly, when you call predict or transform on the pipeline, 
    # it will apply the transformations and predictions in the order specified.
    for _ in range(num_bootstrap_samples):

        indices = np.random.choice(np.arange(0, len(X_train)), len(X_train), replace=True)

        # Get one set of bootstrap sample data
        X_bootstrap= X_train.iloc[indices]
        y_bootstrap= y_train.iloc[indices]
        
        # Create a new model instance of the same type as 'best_model'
        bootstrap_model = clone(best_model)
        # Fit the new model to the bootstrapped sample
        bootstrap_model.fit(X_bootstrap, y_bootstrap)
        # Make predictions on the test data using the new model
        predictions_sample = bootstrap_model.predict(X_test)
        # Append predictions to the list
        bootstrap_predictions.append(predictions_sample)

    # Compute prediction intervals 
    lower_bound = np.percentile(bootstrap_predictions, (1 - confidence) * 100 / 2, axis=0)
    upper_bound = np.percentile(bootstrap_predictions, (1 + confidence) * 100 / 2, axis=0)
    prediction_intervals = np.column_stack((lower_bound, upper_bound))
    # print(prediction_intervals)
    
    # 6.2  Get the top important features
    # 1) Extract the feature names from the polynomial features
    poly_features = best_model.named_steps['poly_features']
    column_names = X_train.columns.tolist() 
    feature_names = poly_features.get_feature_names_out(input_features=column_names)
    
    # Below can be omitted when include_bias=False, feature_names won't contain const
    # Remove 'x0', 'x1' prefix from feature names
    feature_names = [name.replace('x0', 'x1').replace('x1', 'x2') for name in feature_names]

    # 2) Get feature importances or coefficients
    importances = None
    model=best_model.named_steps[f'{model_step_name}']
    print(f"Getting feature importances from model: '{model}'")
    # In scikit-learn, the feature_importances_ attribute is typically used for tree-based models 
    # like Random Forests, Gradient Boosting Machines, etc. 
    # These models do not have a concept of a constant term or intercept like linear models do. 
    # Therefore, we won't find a constant term in the feature importances.
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    elif hasattr(model, 'coef_'):
        importances = np.abs(model.coef_)
    elif hasattr(model, 'dual_coef_'):  # Check for dual coefficients
        importances = np.abs(model.dual_coef_)
    elif hasattr(model, 'params'):  # Assuming 'params' contains the coefficients
        importances = np.abs(model.params)  
    elif hasattr(model, 'tvalues'):
        importances = np.abs(model.tvalues)
    else:
        print("No feature importances available. Plotting for all features...")
    
    top_features = None
    if len(importances) > 0:
        
        num_original_features = X_train.shape[1]
        print(f"Originally, X_train contains {num_original_features} columns.")
        print(f"after fitting the model, features become {len(feature_names)},")
        print(f" and the feature importances contain {len(importances)} values.")
  
        if len(importances) == len(feature_names):
            # Reshape importances array if necessary
            importances = np.ravel(importances)

        else:
            # Calculate the interaction terms from the feature_names 
            # Importances are just importances for the original columns from X_train
            importances_original = importances
            importances_interaction = []

            for name in feature_names[num_original_features:]:
                # Split the term into components if there are spaces (' ')
                components = name.split(' ')
                importance_interaction=1
                for component in components:
                    if '^' in component:
                        base, power =component.split('^')
                    else: 
                        base =component
                        power =1 

                    base_index= feature_names.index(base)
                
                    importance_component = importances_original[base_index] ** int(power)
                    
                    importance_interaction *= importance_component
                # Append the importance of the interaction term to the list
                importances_interaction.append(importance_interaction)

            # Combine the importances for original columns and interaction terms
            importances = np.concatenate((importances_original, importances_interaction))


        # Create a DataFrame to sort and display feature importances

        feature_importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importances
        })
        # Sort the features by importance
        feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

        # Get the top N important features
        top_features = feature_importance_df['feature'].head(top_importances).values
    else: 
        top_features=feature_names
    print(f"top_features: {top_features}")

    # 6.3 Get the full equation from the pipeline
    # As 6.2 does not involve any const / intercept due to include_bias =False:
    # Note: For models in scikit-learn that do not have a built-in intercept_ attribute, such as tree-based models 
    # (e.g., DecisionTreeRegressor, RandomForestRegressor) or support vector machines (SVM), 
    # there isn't a direct attribute that represents the intercept value like in linear models.
    # Only get intercept for linear regression model
    intercept =0
    try:
        intercept = model.intercept_
    except:
        print("No intercept for this model, as include_bias is set to be False.")
    # Construct the full equation
    full_equation = f"{target_variable} = {intercept:.2f}"
    for feature_name, importance in zip(feature_importance_df['feature'], feature_importance_df['importance']):
        full_equation += f" + {importance:.2f} * {feature_name}"
    print(full_equation)
    
    return best_model_name, degree , top_features, prediction_intervals

In [6]:
# Model evaluation
# Subfunctions to visualize performance metrics
def plot_all_metrics(best_model_name,  X,y, testsize, top_features, degree, prediction_intervals,model_step_name, random_seed=42, param_name=None, param_range=None):
    # 1. Split the data using the same random_seed
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testsize, random_state=random_seed)
    
    # Define metrics to plot
    metrics_to_plot = [
        (plot_predicted_vs_actual, 'Predicted vs. Actual Values'),
        (plot_learning_curve, 'Learning Curve'),
        (plot_residual_plot, 'Residual Plot'),
        (plot_studentized_residuals, 'Studentized Residuals vs. Fitted Values'),
        (plot_standardized_residuals, 'Standardized Residuals'),
        (plot_validation_curve, 'Validation Curve'),
        (plot_poly_degree_vs_mse, 'Poly Degree vs. mse'),
        (plot_bias_variance_mse,'Poly Test bias Variance'),
        (plot_prediction_intervals, 'Prediction Intervals'),
        (plot_prediction_actual, 'Predicted vs. Actual, Scatter')
        
    ]

    # Below are not appropriate for Regression models:
    # Confusion Matrix 
    # ROC Curve 
    # Precision-Recall Curve

    # Below are better based on train data rather than test data:
    # Learning Curve,
    # Validation Curve,
    # Bias-Variance
    # Prediction Intervals
    # Studentized Residuals
    # Standardized Residuals

    # 2. Calculate y_pred, residuals for plotting
    # Before prediction, we need to load the pipeline
    pipeline = load(f'{best_model_name}.joblib')

    # 3. Create the figure and axes
    num_metrics = len(metrics_to_plot)
    num_cols = 3
    num_rows = (num_metrics + num_cols - 1) // num_cols
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(6.4*num_cols, 4.8*num_rows))

    # Adjust the spacing between subplots
    plt.subplots_adjust(wspace=1, hspace=1) 

    # Flatten the axes array
    axes = axes.flatten()

    # Loop over the metrics and titles, plotting each one
    for i, (plot_function, title) in enumerate(metrics_to_plot):
        # Plot validation curve only if degree > 0
        if title == 'Validation Curve':
            if degree >0:
                param_name = 'poly_features__degree'
                param_range = np.arange(1,6)
                plot_function(pipeline, X_train, y_train, param_name, param_range, ax=axes[i])
        elif title == 'Poly Degree vs. mse' or title =='Poly Test bias Variance':
            if degree >0: 
                param_range = np.arange(1,6)
                plot_function(pipeline, X_train, y_train, X_test, y_test, param_range, model_step_name, ax=axes[i])
        # Plot other metrics directly
        elif 'Learning' in title:
            plot_function(pipeline, X_train, y_train, ax=axes[i], cv=5)
        elif title == "Prediction Intervals":
            plot_function(pipeline, X_train, y_train, X_test, y_test, prediction_intervals, best_model_name, ax= axes[i])
        elif title in ("Standardized Residuals", "Studentized Residuals vs. Fitted Values"):
            plot_function(pipeline, X_train, y_train, ax=axes[i])
  
        else:
            plot_function(pipeline, X_test, y_test, best_model_name, ax=axes[i] )

    # Hide any empty grid cells
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

def plot_predicted_vs_actual(pipeline, X_test, y_test, best_model_name, ax):
    """
    Plot the predicted vs. actual values.

    Parameters:
    model (object): The trained regression model.
    X_test (array-like): The test data features.
    y_test (array-like): The actual target values for the test data.
    ax (matplotlib axes): The axes to plot the figure.
    """
    y_pred=pipeline.predict(X_test)

    # Below is to give the plot a 45 degree line
    # Define the range for the plot
    min_val = min(min(y_test), min(y_pred))
    max_val = max(max(y_test), max(y_pred))
    ax.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--')

    ax.scatter(y_test, y_pred)
    ax.set_xlabel("Actual y Values")
    ax.set_ylabel("Predicted y Values")
    ax.set_title("Predicted vs. Actual Values\nsk-learn") 

def plot_learning_curve(pipeline, X_train, y_train, ax, cv=None):
    train_sizes, train_scores, test_scores = learning_curve(pipeline, X_train, y_train, cv=cv, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10))
    
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    
    ax.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training mse')
    ax.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
    ax.plot(train_sizes, test_mean, color='green', linestyle='--', marker='s', markersize=5, label='Validation mse')
    ax.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
    ax.grid(True)
    ax.set_xlabel('Number of training samples')
    ax.set_ylabel('Mean Square Error')
    ax.legend(loc='lower right')
    ax.set_title('Learning Curve\nsk-learn')

def plot_residual_plot(pipeline, X_test, y_test, best_model_name, ax):
    y_pred=pipeline.predict(X_test)
    residuals = y_test - y_pred
    ax.scatter(y_pred, residuals)
    ax.axhline(y=0, color='red', linestyle='--')
    ax.set_title('Residual vs. Fitted Values\nsk-learn')
    ax.set_xlabel('Predicted Values')
    ax.set_ylabel('Residuals')

    # in Statsmodels library, we can use below to create plots directly:
    # fig,ax = plt.subplots(1,1, figsize=(10, 6.18))
    # sm.graphics.plot_regress_exog(results, 'horsepower', fig=fig)
    # plt.title('Residuals vs. Fitted Values')

# Before we plot for studentized_residuals or standardized_residuals
# We need to make sure these residuals are calculated based on train data or test data
# Based on train data, the residuals will be more stable especially when test data is in a small amount
# Based on test data, the residuals may not capture the variability of the entire dataset??
# Finally, I decided to use train data for these 2 plots
# If the analysis focuses on new data predictions, change X_train --> X_test

def plot_studentized_residuals(pipeline, X_train, y_train, ax):
 
    y_pred_train=pipeline.predict(X_train)
    residuals_train = y_train - y_pred_train

    # Calculate studentized residuals
    # Hat matrix
    hat_matrix = np.dot(X_train, np.linalg.inv(np.dot(X_train.T, X_train))).dot(X_train.T)
    leverage = np.diag(hat_matrix)
    
    # Calculate the studentized residuals
    std_residuals = np.std(residuals_train)
    studentized_residuals = residuals_train / (std_residuals * np.sqrt(1 - leverage))

    ax.scatter(y_pred_train, studentized_residuals, alpha=0.8)
    ax.axhline(0, color='blue', linestyle='--')
    ax.set_xlabel('Fitted Values')
    ax.set_ylabel('Studentized Residuals')
    ax.set_title('Studentized Residuals vs. Fitted Values\nsk-learn')
    

def plot_standardized_residuals(pipeline, X_train, y_train, ax):
    # Scale Location Plot
    # plot the square root of the standardized residuals against the fitted values 
    # to check for homoscedasticity.
    # Below .fittedvalues and .resid is from sm lib
    # y_pred = model.fittedvalues
    # residuals = model.resid
    # Below is to calculate y_pred and residuals using sk-learn lib
    y_pred_train= pipeline.predict(X_train)
    residuals_train = y_train - y_pred_train
    std_residuals = np.std(residuals_train)
    # Standardize residuals
    standardized_residuals = residuals_train / std_residuals
    
    sqrt_standardized_residuals = np.sqrt(np.abs(standardized_residuals))
    ax.axhline(np.mean(sqrt_standardized_residuals), color='red', linestyle='-')
    ax.scatter(x=y_pred_train, y=sqrt_standardized_residuals)
    ax.axhline(0, color='blue', linestyle='--')
    ax.set_xlabel('Fitted Values')
    ax.set_ylabel('Sqrt(|Standardized residuals|)')
    ax.set_title('Scale-Location Plot\nsk-learn')


def plot_validation_curve(pipeline, X_train, y_train, param_name, param_range, ax):
    """
    Plot validation curve to visualize the effect of model flexibility on performance metrics.

    Parameters:
    model (object): The regression model.
    X_train (array-like): The training data features.
    y_train (array-like): The target values for the training data.
    param_name (str): The name of the hyperparameter to vary.
    param_range (array-like): The range of values for the hyperparameter.
    ax (matplotlib.axes.Axes, optional): The axes to plot the validation curve. If not provided, a new figure and axes will be created.

    Returns:
    ax (matplotlib.axes.Axes): The axes containing the validation curve plot.
    """
    train_scores, test_scores = validation_curve(
        pipeline, X_train, y_train, param_name=param_name, param_range=param_range,
        scoring="neg_mean_squared_error", cv=5, n_jobs=-1
    )

    train_mean = -np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = -np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    if ax is None:
        fig, ax = plt.subplots()

    ax.plot(param_range, train_mean, color="blue", marker="o", markersize=5, label="Training MSE")
    ax.fill_between(param_range, train_mean - train_std, train_mean + train_std, alpha=0.15, color="blue")
    ax.plot(param_range, test_mean, color="green", linestyle="--", marker="s", markersize=5, label="Validation MSE")
    ax.fill_between(param_range, test_mean - test_std, test_mean + test_std, alpha=0.15, color="green")

    ax.set_xlabel(param_name)
    ax.set_ylabel("Mean Squared Error")
    ax.set_title("Validation Curve\nsk-learn")
    ax.legend(loc="best")

# Define a function to create the pipeline with a given degree
def create_pipeline(degree, regressor):
    return make_pipeline(
        PolynomialFeatures(degree=degree),
        regressor
    )
def plot_poly_degree_vs_mse(pipeline, X_train, y_train, X_test, y_test, param_range, model_step_name, ax):
    
    train_mse = []
    test_mse = []
    regressor = pipeline.named_steps[f"{model_step_name}"]
    for degree in param_range:
        
        new_pipeline=create_pipeline(degree, regressor)
        # Fit the model
        new_pipeline.fit(X_train, y_train)
        
        # Predictions
        y_train_pred = new_pipeline.predict(X_train)
        y_test_pred = new_pipeline.predict(X_test)
        
        # Calculate MSE for training and test sets
        train_mse.append(mean_squared_error(y_train, y_train_pred))
        test_mse.append(mean_squared_error(y_test, y_test_pred))

    # Plotting
    ax.plot(param_range, train_mse, label='Training MSE')
    ax.plot(param_range, test_mse, label='Test MSE')
    ax.set_xlabel('Degree of Polynomial')
    ax.set_ylabel('Mean Squared Error')
    ax.set_title('Training and Test MSE vs. Polynomial Degree\nsk-learn')
    ax.legend()
    ax.grid(True)

def calculate_bias_variance(pipeline, X_train, y_train, X_test, y_test):
    # Fit the model
    pipeline.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)
    
    # Bias and variance calculation
    bias = np.mean((y_test - (y_test_pred)) ** 2)
    variance = np.mean((y_test_pred - np.mean(y_test_pred)) ** 2)
    
    # Test MSE calculation
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    return test_mse, bias, variance

def plot_bias_variance_mse(pipeline, X_train, y_train, X_test, y_test, degree_range, model_step_name, ax):
    test_mse_list = []
    bias_list = []
    variance_list = []
    regressor = pipeline.named_steps[f'{model_step_name}']
    for degree in degree_range:
        # Create polynomial features and linear regression model
        new_pipeline = create_pipeline(degree, regressor)
        
        # Calculate test MSE, bias, and variance
        test_mse, bias, variance = calculate_bias_variance(new_pipeline, X_train, y_train, X_test, y_test)
        
        # Append to lists
        test_mse_list.append(test_mse)
        bias_list.append(bias)
        variance_list.append(variance)

    # Plotting
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(degree_range, test_mse_list, label='Test MSE', marker='o',linewidth=5)
    ax.plot(degree_range, bias_list, label='Bias^2', marker='o')
    ax.plot(degree_range, variance_list, label='Variance', marker='o')
    ax.set_xlabel('Degree of Polynomial Features')
    ax.set_ylabel('Mean Square Error')
    ax.set_title('Bias-Variance Tradeoff\nsk-learn')
    ax.legend()
    ax.grid(True)

def plot_prediction_intervals(pipeline, X_train, y_train, X_test, y_test, prediction_intervals, best_model_name, ax):
    # Prediction intervals provide a range of values within 
    # which future observations are expected to fall, with a certain level of confidence. 
    # Bootstrapping is a resampling method that can be used to estimate the uncertainty of a statistical estimate, including prediction intervals. 
    # It involves repeatedly resampling the data with replacement 
    # and fitting the model to each resampled dataset to generate a distribution of predictions.
    pipeline = load(f'{best_model_name}.joblib')
    # Generate predictions on the test set
    y_pred = pipeline.predict(X_test)
    
    # Plot the 'actual' predicted values
    ax.scatter(np.arange(len(X_test)), y_pred, label='Predictions')
    
    # Plot the prediction intervals
    ax.fill_between(np.arange(len(X_test)), prediction_intervals[:, 0], prediction_intervals[:, 1], alpha=0.3, color='orange', label=f'{int(confidence * 100)}% Prediction Intervals')
    
    # Add labels and title
    ax.set_xlabel('Sample Index') # index for each data point , or index for each row
    ax.set_ylabel('Predicted Value')
    ax.set_title('Prediction Intervals\nsk-learn')
    ax.legend()

def plot_prediction_actual(pipeline, X_test, y_test,best_model_name, ax):
    pipeline = load(f'{best_model_name}.joblib')
     # Generate predictions for the test set
    y_pred = pipeline.predict(X_test)
    
    # Plot the actual values
    ax.scatter(np.arange(len(X_test)), y_test, label='Actual Values')
    
    # Plot the actual values
    ax.scatter(np.arange(len(X_test)), y_pred, label='Predicted Values')


    # Add labels and title
    ax.set_xlabel('Data Points') 
    ax.set_ylabel('Predicted Value')
    ax.set_title('Prediction vs. Actual\nsk-learn')
    ax.legend()
    # del pipeline # to free up the memory
# Subfunction to visualize dataset according to data characteristics
def visualize_dataset(df):
    print("Starting to visualize dataset...")
    sns.pairplot(df.sample(frac=0.1, random_state=42), hue='target', palette='bwr')
    plt.title("Pairplot of Dataset")
    plt.show()

    corr = df.corr()
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5)
    plt.title('Correlation Heatmap')
    plt.show()

    df.describe().transpose().plot(kind='hist', bins=20, subplots=True, layout=(5, 6), figsize=(20, 15))
    plt.suptitle("Distributions of Features", y=1.02)
    plt.tight_layout()
    plt.show()


In [7]:
# 2. Subfunctions to load the dataset
def find_filepath(name):
    for root, dirs, files in os.walk('..'):
        for file in files:
            base, extension = os.path.splitext(file)
            if extension.lower() in ('.txt', '.csv'):
                if os.path.splitext(name)[1]:  # If name has an extension
                    if file.lower() == name.lower():  # Compare filename with name
                        file_path = os.path.join(root, file)
                        if os.path.isfile(file_path):
                            return file_path
                else:  # If name does not have an extension
                    if base == os.path.splitext(name)[0]:  # Compare base part of filename with name
                        file_path = os.path.join(root, file)
                        if os.path.isfile(file_path):
                            return file_path
    return None

def get_dataframe(original_data, target_variable=None):
    if isinstance(original_data, pd.DataFrame):
        return original_data
    else:
        try:
            data = pd.DataFrame(original_data.data, columns=original_data.feature_names)
            data[target_variable] = original_data.target
            return data
        except:
            data = pd.DataFrame(original_data)
            return data

def capitalize_first_letter(word):
    # Capitalize the first letter and convert the rest of the word to lowercase
    return word[:1].upper() + word[1:].lower()

def load_dataset(name):
    try: 
        
        capitalized_name = capitalize_first_letter(name)
        target_variable = None
        data = load_data(f"{capitalized_name}")
        
        return get_dataframe(data, target_variable)
    except:
        if name == 'california_housing':
            target_variable = 'MedHouseVal'
            original_data = fetch_california_housing()
            return get_dataframe(original_data, target_variable), target_variable
        else: 
            return 'No dataset named ' + capitalized_name


In [ ]:
# Main Script
if __name__ == "__main__":

    # 2. Load the Dataset (Boston dataset)
    try:
        dataset_name = 'boston'
        # dataset_name = "auto"
        data = load_dataset(dataset_name)
    except Exception as e:
        print(f"An error occurred while loading the dataset: {e}")
        exit()

    # 3. Define X and Y
    drop_column_list = [target_variable]
    y= data[target_variable]
    X= data.drop(columns= drop_column_list)

    # Specify the degree of the polynomial and test size
    degree_list = [1,2]  # Replace with desired degree or obtain from user input
    testsize = 0.2  # Replace with desired test size or obtain from user input
    
    # 4. Train and Evaluate Regression Models and get the best model
    confidence = 0.95
    random_seed=42
    num_bootstrap_samples=1
    top_importances = 10
    model_step_name="regression_model"
    best_model_name, degree ,top_features, prediction_intervals=fit_regression_models(X,y,testsize,degree_list, num_bootstrap_samples, confidence, random_seed,top_importances,model_step_name)

    # 5. Visualize Model Performance 
    plot_all_metrics(best_model_name, X,y, testsize, top_features, degree, prediction_intervals, model_step_name, random_seed)

    # 6. Predictions on new data
    # Predict single observation:
    if best_model is not None:
        sample_data = X_test.iloc[[0]]  # Example: Use the first instance from the test set
        prediction = best_model.predict(sample_data)
        print(f"Prediction for sample data: {prediction}")
    else:
        print("No best regression model found, cannot make predictions.")
###### scikit-learn/regression ######